In [1]:
# ----------------------- import libraries -------------------

import nltk
import hazm
import sklearn as sk
import pandas as pd
import numpy as np
import hazm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from nltk.classify.scikitlearn import SklearnClassifier

In [2]:
with open('data/train.csv') as f:
    data = f.readlines()
data = [x.strip() for x in data]

labels = [x.split('\t')[-2] for x in data]
label_id = [x.split('\t')[-1] for x in data]
texts = [x.split('\t')[1:-2] for x in data]
data_train = pd.DataFrame({'text':texts,'label':labels,'label_id':label_id})
data_train['text'] = data_train['text'].apply(lambda x: ' '.join(x))
# data_train['text'] = data_train['text'].apply(lambda x: hazm.word_tokenize(x))
print(type(data_train['text'][0]))

<class 'str'>


,text,label,label_id
0,,label,label_id
1,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
2,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
3,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
4,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
...,...,...,...
56696,یک تیکه کم فرستاده بودن و با تماس من در کمترین...,HAPPY,0
56697,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...,HAPPY,0
56698,مثل همیشه عالی، من چندمین باره سفارش میدم و هر...,HAPPY,0
56699,دلستر استوایی خواسته بودم اما لیمویی فرستادند,HAPPY,0


In [3]:
from __future__ import unicode_literals
from hazm import *
normalizer = Normalizer()
data_train['text'] = data_train['text'].apply(lambda x: normalizer.normalize(x))

In [4]:
# data_train['text'] = data_train['text'].apply(lambda x: hazm.word_tokenize(x))
data_train['text'] = data_train['text'].str.replace('۰','0')
data_train['text'] = data_train['text'].str.replace('۱','1')
data_train['text'] = data_train['text'].str.replace('۲','2')
data_train['text'] = data_train['text'].str.replace('۳','3')
data_train['text'] = data_train['text'].str.replace('۴','4')
data_train['text'] = data_train['text'].str.replace('۵','5')
data_train['text'] = data_train['text'].str.replace('۶','6')
data_train['text'] = data_train['text'].str.replace('۷','7')
data_train['text'] = data_train['text'].str.replace('۸','8')
data_train['text'] = data_train['text'].str.replace('۹','9')

In [5]:
data_train

,text,label,label_id
0,,label,label_id
1,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
2,قرار بود 1 ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
3,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
4,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
...,...,...,...
56696,یک تیکه کم فرستاده بودن و با تماس من در کمترین...,HAPPY,0
56697,عالی بود همه چیز ممنونم پیک هم خیلی مرتب و به ...,HAPPY,0
56698,مثل همیشه عالی، من چندمین باره سفارش میدم و هر...,HAPPY,0
56699,دلستر استوایی خواسته بودم اما لیمویی فرستادند,HAPPY,0


In [6]:
# cleaning text
processed = data_train['text'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 ' emailaddress ',regex=True)
processed = processed.str.replace(r'(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&:/~\+#]*[\w\-\@?^=%&/~\+#])?',
                                  ' webaddress ',regex=True)
processed = processed.str.replace(r'(تومان|ریال|دلار|\$|تومن|هزار|میلیون|یورو|قرون|دیر|سرد|افت|فاجعه|نمیخوام|نمیخرم|نمی خرم|نمیکنم|متاسفانه|احمقانه)', ' moneysymb ',regex=True)

processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  ' phonenumbr ',regex=True)
processed = processed.str.replace(r'\d+(\.\d+)?', ' numbr ',regex=True)
processed = processed.str.replace(r'(ده|بیست|صد|سی|هزار|چهل|پنجاه|شصت|هفتاد|هشتاد|نود|دو|سه|چهار|پنج|شش|هفت|هشت|یازده|دوازده|سیزده|چهارده)', ' numbr ',regex=True)
processed = processed.str.replace(r'(پوزنده|شونزده|هفده|هیفده|هجده|نوزده|میلیارد)', ' numbr ',regex=True)


processed = processed.str.replace(r'[^\w\d\s]', ' ',regex=True)
processed = processed.str.replace(r'(نیاوردین|نخریدم|نمیخردم|نمیخوام|نمیشنوم|بد|پایین|گرون|ناراضی|نبود|نیست)', ' BAD ',regex=True)
processed = processed.str.replace(r'(افتضاح|فاجعه|مسخره|چرت|گند|نداشت|افت|کثیف|خراب|زشت|نپخته|سوخته|نیست|نیاوردین|نبود|نشده|کثیف)', ' BAD ',regex=True)
processed = processed.str.replace(r'[^\w\d\s]', ' ',regex=True)

processed = processed.str.replace(r'(بد|شکسته|نمیخرم|نمی خرم|نمیخوام|نمی خوام|نمیشه|نمی شه|نیست|ناراضی|نکردین|نشده|نخواهد|نشد|کهنه|نکرد|نمیکنه|نمی کنه|متاسفانه)', ' BAD ',regex=True)
processed = processed.str.replace(r'(نداره|کم|سوخته|کم|سیاه|خراب|قدیمی|کهنه|ماسیده|فاجعه|نداشتن|دقت|خمیر|شفته|نیستم)', ' BAD ',regex=True)
processed = processed.str.replace(r'^\s+|\s+?$|,', '',regex=True)

processed = processed.str.lower()

In [7]:
data_train['text'] = processed

In [8]:

ps = Lemmatizer()
word_tokenizer = WordTokenizer()
processed = processed.apply(lambda x: [ps.lemmatize(word) for word in word_tokenizer.tokenize(x)])
stopwords = set(stopwords_list())
alphabet = ['ا','آ','ب','پ','ت','ث','ج','چ','ح','خ','د','ذ','ر','ز','ژ','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ک','گ','ل','م','ن','و','ه','ی','ار','ای']
processed = processed.apply(lambda x:[
    term for term in x if term not in stopwords and term not in alphabet])
# processed = processed.apply(lambda x: [ps.lemmatize(term) for term in x])

In [9]:
all_words = []
for i in processed:
  for j in i:
    all_words.append(j)

In [10]:
all_words = nltk.FreqDist(all_words)
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(40)))

Number of words: 19830
Most common words: [('numbr', 83819), ('بود#باش', 60660), ('bad', 40757), ('moneysymb', 15651), ('غذا', 15407), ('کیفیت', 12507), ('سفارش', 9644), ('پیتزا', 8063), ('کرد#کن', 6113), ('واقعا', 5725), ('ممنون', 5498), ('اصلا', 5429), ('ارسال', 5065), ('خوشمزه', 5059), ('طعم', 4776), ('شد#شو', 4728), ('مرغ', 4667), ('ها', 4536), ('مزه', 4049), ('یه', 4041), ('پیک', 4019), ('تازه', 3990), ('قیمت', 3842), ('داد#ده', 3345), ('تحویل', 3296), ('نان', 3281), ('سس', 3264), ('گرم', 3236), ('بسته', 3134), ('حجم', 3061), ('رستوران', 3039), ('سان', 2957), ('زمین', 2939), ('داشت#دار', 2854), ('شیرینی', 2804), ('#هست', 2684), ('نون', 2640), ('تر', 2627), ('یچ', 2569), ('اسنپ', 2467)]


In [ ]:
word_features = list(all_words.keys())[:1500]

In [ ]:
def find_features(message):
    words = message
    features = []
    for word in word_features:
        features.append(int(word in words))
    return features

In [ ]:
featuresets = [find_features(text) for text in processed.iloc[1:]]

In [ ]:
X,Xt, Y,Yt = sk.model_selection.train_test_split(featuresets,data_train['label_id'][1:], test_size = 0.25, random_state=1)
X, Xt, Y, Yt = np.array(X), np.array(Xt), np.array(Y), np.array(Yt)


In [ ]:
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(),
    MultinomialNB(),
    SVC()
]
for clf in classifiers:
    clf.fit(X, Y)
    y_pred = clf.predict(Xt)
    print(clf.__class__.__name__ + ':', accuracy_score(Yt, y_pred)*100)